In [55]:
# Owen Wichiencharoen's standard Python Imports:

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

import matplotlib.pyplot as plt
%matplotlib inline


### REGRESSION PACKAGES
# from scipy import stats
# import statsmodels.formula.api as smf
# import statsmodels.api as sm
# import sklearn.linear_model as lm

### DATASET PACKAGES
# from sklearn.cross_validation import train_test_split, cross_val_score
# from sklearn import datasets, metrics
from sklearn.preprocessing import LabelEncoder

### TREE/RANDOMFOREST PACKAGES
# from sklearn.tree import DecisionTreeClassifier, export_graphviz
# from sklearn.ensemble import RandomForestClassifier
# import pydot
# from os import system
# from sklearn.externals.six import StringIO
# from IPython.display import Image

### OTHER PACKAGES
#import itertools
#import pandas_datareader.data as pdweb
#from pandas_datareader.data import DataReader
#from datetime import datetime
#from io import StringIO
from IPython.display import Image

## Important notes from Kaggle

https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums

- not all events have location (latitude/longitude), and you can treat (0,0),(1,0),(0,1),(1,1) lat/long as NaN.
- Some device_ids have duplicates (mappable to two brands/models) - the user probably upgraded his or her phone.
- ...

## Data Descriptions

- gender_age_train.csv, gender_age_test.csv - the training and test set group: this is the target variable you are going to predict
- events.csv, app_events.csv - when a user uses TalkingData SDK, the event gets logged in this data. Each event has an event id, location (lat/long), and the event corresponds to a list of apps in app_events. timestamp: when the user is using an app with TalkingData SDK
- app_labels.csv - apps and their labels, the label_id's can be used to join with label_categories
- label_categories.csv - apps' labels and their categories in text
- phone_brand_device_model.csv - device ids, brand, and models

In [15]:
Image(url='http://i.imgsafe.org/132c9be39b.png') 

In [2]:
raw_events = pd.read_csv('../../kaggle_data/events.csv')

In [3]:
raw_label_categories = pd.read_csv('../../kaggle_data/label_categories.csv')
raw_phone_brand_device_model = pd.read_csv('../../kaggle_data/phone_brand_device_model.csv',encoding='utf-8')
raw_gender_age_train = pd.read_csv('../../kaggle_data/gender_age_train.csv')
raw_gender_age_test = pd.read_csv('../../kaggle_data/gender_age_test.csv')
raw_app_events = pd.read_csv('../../kaggle_data/app_events.csv')
raw_app_labels = pd.read_csv('../../kaggle_data/app_labels.csv')

#### EVENTS TABLE (main)

In [32]:
print('NUMBER OF ROWS:', raw_events.shape[0])

print('\nNULL VALUES:')
print(raw_events.isnull().sum())

print("\nUNIQUE VALUES:")
for c in raw_events.columns:
    print('{}: {}'.format(c, raw_events[c].nunique()))

raw_events.head(3)

NUMBER OF ROWS: 3252950

NULL VALUES:
event_id     0
device_id    0
timestamp    0
longitude    0
latitude     0
dtype: int64

UNIQUE VALUES:
event_id: 3252950
device_id: 60865
timestamp: 588125
longitude: 3588
latitude: 3086


,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70


#### PHONE_BRAND_DEVICE_MODEL TABLE

In [35]:
print('NUMBER OF ROWS:', raw_phone_brand_device_model.shape[0])

print('\nNULL VALUES:')
print(raw_phone_brand_device_model.isnull().sum())

print("\nUNIQUE VALUES:")
for c in raw_phone_brand_device_model.columns:
    print('{}: {}'.format(c, raw_phone_brand_device_model[c].nunique()))

raw_phone_brand_device_model.head(3)

NUMBER OF ROWS: 187245

NULL VALUES:
device_id       0
phone_brand     0
device_model    0
dtype: int64

UNIQUE VALUES:
device_id: 186716
phone_brand: 131
device_model: 1599


,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4


In [21]:
raw_phone_brand_device_model['device_id'].value_counts().sort_values(ascending=False)[:4]

 9218059356897525342    2
 4781989393190512097    2
 5435517613682152801    2
-7525913571507064767    2
Name: device_id, dtype: int64

In [50]:
# Expected device_ids to be unique. Looks like there are duplicates

dupes = raw_phone_brand_device_model.groupby('device_id').size()
dupes = dupes[dupes>1]
print('NUMBER OF DUPLICATES:', dupes.shape[0])

print('\nWHAT N OF DUPLICATES:')
dupes.value_counts()

NUMBER OF DUPLICATES: 529

WHAT N OF DUPLICATES:


2    529
dtype: int64

In [51]:
# Are these duplicates exactly the same (we can drop them) or are they different?

dupes = raw_phone_brand_device_model.loc[raw_phone_brand_device_model['device_id'].isin(dupes.index)]
firstd = dupes.groupby('device_id').first() # nrows = 529
lastd = dupes.groupby('device_id').last() # nrows = 529

diff = (firstd != lastd).sum(axis=1).nonzero()
pd.concat((firstd.iloc[diff], lastd.iloc[diff]),axis=1)

,phone_brand,device_model,phone_brand,device_model
device_id,,,,
-7297178577997113203,华为,荣耀畅玩5X,华为,荣耀3C
-7059081542575379359,LG,Nexus 5,魅族,魅蓝Note 2
-6590454305031525112,小米,MI 3,华为,荣耀6 Plus
-5269721363279128080,三星,Galaxy Core Advance,小米,MI 3
-3004353610608679970,酷派,5891,酷派,7296
5245428108336915020,魅族,MX4,小米,MI One Plus


In [52]:
# Looks like 6 devices with duplicate rows have different values for brand and model.

# Are they in train or in test?

dev = _.index
print("in train: ",sum(raw_gender_age_train['device_id'].isin(dev)))
print("in test: ",sum(raw_gender_age_test['device_id'].isin(dev)))

in train:  1
in test:  5


In [53]:
# Only 6 rows are weird. Let's just drop all duplicates

phone_brand_device_model = raw_phone_brand_device_model.drop_duplicates('device_id', keep='first')
del raw_phone_brand_device_model

In [54]:
# KAGGLE FORUM POINTS OUT THAT SOME MODEL NAMES CAN BELONG TO DIFFERENT BRANDS!!!

x = phone_brand_device_model.groupby('device_model')['phone_brand'].apply(pd.Series.nunique)
x.value_counts()

1    1545
2      43
3       8
4       3
Name: phone_brand, dtype: int64

In [ ]:
# Use the same trick as we would in excel:  create a Brand-Model column by concatenating them.

lebrand = LabelEncoder().fit(phone_brand_device_model.phone_brand)
phone['brand'] = lebrand.transform(phone_brand_device_model.phone_brand)
m = phone.phone_brand.str.cat(phone_brand_device_model.device_model)
lemodel = LabelEncoder().fit(m)
phone['model'] = lemodel.transform(m)